# Models

And now to try and predict.

In [2]:
# Load in the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

Now to set up the data again. As before, refer to notebooks 1 and 2 for more information.

In [3]:
# Set up the data
lines = pd.read_csv('../data/All-seasons.csv')

lines = lines[lines.Season != 'Season']

lines[['Season', 'Episode']] = lines[['Season', 'Episode']].astype('int64')

#main_chars = ['Cartman', 'Stan', 'Kyle', 'Kenny', 'Butters']

lines['is_cartman'] = 0

lines.loc[lines.Character == 'Cartman', 'is_cartman'] = 1

lines.head(3)

,Season,Episode,Character,Line,is_cartman
0,10,1,Stan,"You guys, you guys! Chef is going away. \n",0
1,10,1,Kyle,Going away? For how long?\n,0
2,10,1,Stan,Forever.\n,0


### Corpus

The corpus is established here, with steps to convert everything to lowercase and remove punctuation from the end of each word.

In [5]:
import re, string

corpus = lines.Line.tolist()

for line in range(len(corpus)):
    corpus[line] = re.sub('\\n', '', corpus[line].rstrip()).lower()
    corpus[line] = " ".join(word.strip(string.punctuation) for word in corpus[line].split())
    
corpus[7:10]

["what's the meaning of life why are we here",
 "i hope you're making the right choice",
 "i'm gonna miss him i'm gonna miss chef and i...and i don't know how to tell him"]

<b>Improving the corpus by removing contractions and lemmatizing words</b>

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import contractions

Here is a loop to expand all contractions:

In [7]:
for line in range(len(corpus)):
    corpus[line] = contractions.fix(corpus[line])
                                  
corpus[7:10]

['what is the meaning of life why are we here',
 'i hope you are making the right choice',
 'I am going to miss him I am going to miss chef and i...and i do not know how to tell him']

Now a function to lemmatize all verbs and nouns:

In [8]:
lem = WordNetLemmatizer()

def lemmatize_lines(line):
    word_list = word_tokenize(line)
    
    word_list = [lem.lemmatize(w, pos='v') for w in word_list]
    
    lem_line = ' '.join([lem.lemmatize(w) for w in word_list])
    
    return lem_line

And finally, a loop using the previous function to execute the lemmatization:

In [9]:
for line in range(len(corpus)):
    corpus[line] = lemmatize_lines(corpus[line])
    
corpus[7:9]

['what be the mean of life why be we here',
 'i hope you be make the right choice']

### Stop words

The list of stop words as determined in notebook 2 needs to instantiated.

In [10]:
sw = ['be', 'you', 'i', 'to', 'the', 'do', 'it',\
        'a', 'we', 'that', 'and', 'have', 'go', 'what',\
        'get', 'of', 'this', 'in', 'on', 'all', 'just',\
        'for', 'he', 'know', 'will', 'but', 'with', 'so',\
        'they', 'now', 'well', "'s", 'guy', 'u', 'come',\
        'like', 'there', 'at', 'would', 'who', 'him',\
        'them', 'his', 'thing', 'where', 'should', 'an',\
        'please', 'maybe', 'their', 'even', 'any', 'than']

### Word vectors and Data splits

The words and the corpus have been preprocessed, now it's time to continue the setup by establishing the word vector and then splitting up the train and test sets. From there, we can tune different models and try to determine the best predictor. We first start with CountVectorizer to establish a basic bag-of-words, combined with a Naive Bayes algorithm for training.<br>
<br>
<b>Establishing the vector</b>

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [12]:
vectorizer = CountVectorizer(stop_words=sw, ngram_range=(1,2))

X = vectorizer.fit_transform(corpus)
y = lines.is_cartman

state = 3

# How many features are there?
X.shape[1]

232654

<b>Splitting the data and training Multinomial Naive Bayes</b><br>
Now the features, X, and the target labels, y, need to be split into test and training sets. Then we can fit the data to a Multinomial Naive Bayes model and cross validate on the training data before checking against the test data.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=state)

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

mnb = MultinomialNB()

cv_scores = cross_val_score(mnb, X_train, y_train, cv=5)

In [15]:
print('Scores: ', cv_scores)
print('Average score: ', np.mean(cv_scores))

Scores:  [0.85963321 0.86013704 0.86102993 0.8612175  0.8597057 ]
Average score:  0.8603446770723983


We would certainly like to see better results, but it's not terrible for the first run.<br>
<br>
The immediate next steps would be to try differt n-gram ranges for the count vectorizer, and different values of *alpha* for MultinomialNB. Let's experiment with n-grams first.

<b>Results for different n-gram ranges: MultinomialNB</b>

| ngram_range | Number of features | Mean cv score |
|-------------|-----------------|----------------------|
|  (1,1)      | 20,939           | 0.685                 |
| (1,2)       | 232,654          | 0.697                 |
| (1,3)       | 536,629          | 0.696                 |
| (1,5)       | 1,060,181         | 0.677                 |

Increasing the n-grams to (1,2) improves the accuracy, while also greatly increasing the features. Anything beyond that only adds features without any added benefit.<br>
<br>
Now we can experiment with values of *alpha* using `GridSearchCV`.

In [16]:
from sklearn.model_selection import GridSearchCV

params = {'alpha': [1.35, 1.4, 1.45, 1.5]}

nb_grid = GridSearchCV(mnb, params, cv=5)

nb_grid.fit(X_train, y_train)

print('Best alpha: ', nb_grid.best_estimator_)
print('Best score: ', nb_grid.best_score_)

Best alpha:  MultinomialNB(alpha=1.35, class_prior=None, fit_prior=True)
Best score:  0.8616950518996271


*Alpha* helps, but only slightly.

### Tf-idf

To try and make further improvements, we can use a weighted tf-idf vectorizer instead of a basic CountVectorizer. Stop words aren't as big of a concern here due to the inherent adjustments, but since we already have a list we may as well use it. Also, because this is a new vectorizer we will have to create new train and test splits.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vect = TfidfVectorizer(stop_words=sw)

X = tf_vect.fit_transform(corpus)
y = lines.is_cartman

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=state)

Now to run the new splits with MultinomialNB and check the scores.

In [20]:
params = {'alpha': [1.0, 1.25, 1.35, 1.4]}

t_nb_grid = GridSearchCV(mnb, params, cv=5)

t_nb_grid.fit(X_train, y_train)

print('Best alpha: ', t_nb_grid.best_estimator_)
print('Best score: ', t_nb_grid.best_score_)

Best alpha:  MultinomialNB(alpha=1.35, class_prior=None, fit_prior=True)
Best score:  0.8624811045046861


Combined with MultinomialNB, the tf-idf vectorizer is a slight improvement compared to CountVectorizer. However, there may be other algorithms better suited to the weighted vectors.

### Trying other models

<b>Random Forest</b><br>
The results so far have been decent, but maybe there are other algorithms that will perform better than Naive Bayes. First, let's try Random Forest, still with the tf-idf vectors. With `oob_score` (out of bag) set to 'True', we use the out-of-bag samples as a sort of validation set.

In [21]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100,\
                            max_features='sqrt',\
                            oob_score=True,\
                            random_state=state,\
                            n_jobs=-1)

rf.fit(X_train, y_train)
print(rf.oob_score_)

0.8628237428197117


The score is pretty in line with all the other results we've seen. There are other parameters we could play with, but the results don't look any more promising than Naive Bayes, so let's try some others.

<b>SVM</b><br>
We can also try Support Vector Machines.

In [22]:
from sklearn.svm import LinearSVC

svm = LinearSVC(max_iter=5000, random_state=3)

params = {'C': [0.1, 0.15, 0.2]}

sv_grid = GridSearchCV(svm, params, cv=5)

sv_grid.fit(X_train, y_train)

print('Best C: ', sv_grid.best_estimator_)
print('Best score: ', sv_grid.best_score_)

Best C:  LinearSVC(C=0.2, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=5000,
          multi_class='ovr', penalty='l2', random_state=3, tol=0.0001,
          verbose=0)
Best score:  0.8664718331149854


Not a huge improvement, but at least we managed to crack the 70% mark.

<b>Logistic Regression</b>

In [31]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=4000, multi_class='multinomial', random_state=3)

params = {'C': [1.0, 1.5, 1.75],
         'solver': ['newton-cg', 'sag']}

log_grid = GridSearchCV(log_reg, params, cv=5)

log_grid.fit(X_train, y_train)

print('Best C: ', log_grid.best_estimator_)
print('Best score: ', log_grid.best_score_)

Best C:  LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=3, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
Best score:  0.8678222311800867


Playing around wih different GridSearch parameters yields some better results, but it seems clear at this point that we've hit a bit of a wall. 